<a href="https://colab.research.google.com/github/zuleyhairmakk/Sentiment-analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dowloading the dataset



In [ ]:
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.keras.utils.get_file("aclImdb_v1",
                         url,
                         untar=True,
                         cache_dir=".",
                         cache_subdir=" ")

84140032/84125825 [==============================] - 3s 0us/step


In [ ]:
import os
dataset_dir=os.path.join(os.path.dirname(dataset),"aclImdb")

In [ ]:
os.listdir(dataset_dir)

['imdb.vocab', 'test', 'README', 'train', 'imdbEr.txt']

In [ ]:
train_dir=os.path.join(dataset_dir , "train")
os.listdir(train_dir)

['neg',
 'pos',
 'labeledBow.feat',
 'urls_unsup.txt',
 'urls_pos.txt',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt']

In [ ]:
sample_file= os.path.join(train_dir, "pos/1181_9.txt")
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [ ]:
remove_dir=os.path.join(train_dir, "unsup")
import shutil
shutil.rmtree(remove_dir)

In [ ]:
## Loading the dataset


In [ ]:
batch_size=32
seed=42
raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed

)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [ ]:
import numpy as np
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [ ]:
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed

)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
test_dir=os.path.join(dataset_dir, "test")
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
)

Found 25000 files belonging to 2 classes.


In [ ]:
import re
import string
def custom_standardization(input_data):
  lowercase=tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')